# Исследование надёжности заёмщиков

**Цель исследования:** установить какие факторы влияют на факт погашения кредита в срок, дать ответы на следующие вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Ход исследованя**: 
1. Обзор данных
2. Предабработка данных
3. Ответ на поставленные вопросы

##  Обзор данных

Импортируем библиотеку `pandas` и прочитаем файл от заказачика:

In [1]:
#файл сохраним в переменной 'bank_df'
#применим конструкцию try-except для отработки ошибок при открытия файла
import pandas as pd
try:
    bank_df = pd.read_csv('/datasets/bank_data.csv')
except:
    bank_df =pd.read_csv('bank_data.csv')
#отключение предупреждений в Юпитере
import warnings
warnings.filterwarnings('ignore') 

Выведем на экран первые 15 строк таблицы  `bank_df`:


In [2]:
display(bank_df.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Изучим общую информацию о таблице:

In [3]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Первичный обзор данных показал, то что в таблице 12 стоблцов и более 2000 строк, типы данных: 
* `int64` 
* `float64` 
* `object`

Согласно документации:
* `children`  — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` —  возраст клиента в годах
* `education` — возраст клиента в годах 
* `education_id` — уровень образования клиента
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В таблце есть недочёты:
1. В столбцых `days_employed` и `total_income` есть пропущенные значения `NaN`
2. В столбце `days_employed` пристуствует аномалия в виде отрицательных и неправдаподобных значений
3. В столбце `education` есть неявные дубликаты, например: `высшее` - `ВЫСШЕЕ` и др.
4. В столбцах `total_income` и `days_employed` используется не самый удачный тип данных, с нецелыми числами, `debt` - некорретный тип               

### Выводы:  <a class="tocSkip">
В исходной таблице есть недочёты в виде пропущенных значений, дубликатов и аномалий и т.п., для ответов на поставленные вопросы в цели задачи данным требуется предобработка

## Предобработка данных

###  Обработка проущенных значений
Откроем первые 10 строк с пропeщенными значениями в столбце `days_employed`:


In [4]:
# для выбора строк с пропущенными значениями примерним метода isna()
bank_df[bank_df['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Посчитаем колличесвто пропущенных значений в столбце `days_employed`: 

In [5]:
# для подсчёта колличеств пропущенных значений воспользуемся функцией len()
len(bank_df[bank_df['days_employed'].isna()]) 

2174

При просмотре таблицы с пропущенными значениями обнаружена закономерность: если есть пропуск, в строке `days_employed`, то и в  `total_income` значение также отсутсвует. проверм эту *гипотезу* повторив аналагичные действия для `total_income`, но выведем последнии 10 строк:

In [6]:
# последние строки выведем методом tail()
bank_df[bank_df['total_income'].isna()].tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21415,0,NaN,54,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,операции с жильем
21423,0,NaN,63,среднее,1,женат / замужем,0,M,пенсионер,0,NaN,сделка с автомобилем
21426,0,NaN,49,среднее,1,женат / замужем,0,F,сотрудник,1,NaN,недвижимость
21432,1,NaN,38,неоконченное высшее,2,Не женат / не замужем,4,F,сотрудник,0,NaN,операции с жильем
21463,1,NaN,35,высшее,0,гражданский брак,1,M,сотрудник,0,NaN,на проведение свадьбы
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля


В этой выборке данная закономерность подтверждается, убедимся в этом, подсчитав колличество пропущенных значений для `total_income`

In [7]:
len(bank_df[bank_df['total_income'].isna()])  

2174

Значения равны, поэтому можем подтвердить гипотезу о том, что строки с проущенными значениями `days_employed` и `total_income` соответсвуют друг другу.

*Данная закономерность объясняется зависимостью ежемесячного дохохода от трудового стажа. Посольку в банковской базе данных используется только официальная информация о доходах, то у человека не имеющего трудового стажа не будет и дохода. (искл. 'пенсионеры', 'предприниматели')*

    

Посчитаем отношение строк с пропущенными значениями к общему колличеству строк: 

In [8]:
# используем фкнкцию len() в числителе и знаменателе для подсчета колличества значений
len(bank_df['days_employed']) / len(bank_df[bank_df['total_income'].isna()])  

9.901103955841766

Строки с пропущенными значениями составляют почти 10% от всего датафрейма, что бы не терять 1/10 части данных удалением, выберем оптимальный способ замены:
* для `days_employed`	- 0, т.к. согласно выдвинутой гипотезе пропущены значения у людей не имеющих трудового стажа
* для `total_income` - медианное значение, т.к. доходы у людей могут сильно различаться

Выведем медиану на экран:

In [9]:
# медиану подчитаем методом .median()
total_income_median = bank_df['total_income'].median()
print(total_income_median)

145017.93753253992


Заполним пропущенные значения, для `days_employed` и `total_income`:

In [10]:
# заменим значения с помощью функции fillna в скобках которой укажем соответствующии для замены значений
bank_df['days_employed'] =  bank_df['days_employed'].fillna(0)
bank_df['total_income'] =  bank_df['total_income'].fillna(total_income_median)


Проверим, на пропуски:

In [11]:
print(len(bank_df[bank_df['days_employed'].isna()]))  
print(len(bank_df[bank_df['total_income'].isna()]))  

0
0


### Обработка анамалий 

#### Обработка анамалий в стобце общего трудового стажа 

Во время предварительного обзора данных обнаружена анамалия в виде отрицательных и неправдоподобно больших значений в столбце хранящим информацию об общем трудовом стаже, постараемся выяснить причину данной анамалии и скорретировать данные

Для начала выясним сколько строк содержат отрицательное значение:

In [12]:
len(bank_df.loc[bank_df['days_employed'] < 0 ,'days_employed'])

15906

Посчитаем процент строк с отрицательными значениями:

In [13]:
(len(bank_df.loc[bank_df['days_employed'] < 0 ,'days_employed']) * 100) / len(bank_df['days_employed'])

73.89547038327527

Имеем почти 74 процента строк с отрицательными значениями, выдвеним предположение о том, что возможна ошибка в занесении данных, попробуем найти закоомерность изучив некоторые из строк:

In [14]:
#display(bank_df.loc[bank_df['days_employed'] < 0].head())
display(bank_df.loc[bank_df['days_employed'] < 0].tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Закономерности от других столбцов не было обнаружено, поэтому делаем вывод о том, что это ошибка занесения данных, которая заключается в том, что минус был добавлен ошибочно, проверим по среднему значению, корректность предоложения:

In [15]:
(bank_df.loc[bank_df['days_employed'] <0, 'days_employed'].mean() / 365)  

-6.4466189917777506

Средний стаж работы 6,5 лет - адекватная цифра которая может соответствовть дейстительности. 

Рассмотрим выборку с положительными значениями:

In [16]:
print(len(bank_df.loc[bank_df['days_employed'] > 0 ,'income_type']))

3445


Посчитаем процент с полдожительными значениями:

In [17]:
print((len(bank_df.loc[bank_df['days_employed'] > 0 ,'days_employed']) * 100) / len(bank_df['days_employed']))

16.004645760743323


Значения с положительным знаком гораздо меньше, постаремся найти закономерность, изучим строки:

In [18]:
#display(bank_df.loc[bank_df['days_employed'] > 0].head())
display(bank_df.loc[bank_df['days_employed'] > 0].tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем


Обнаружена закономерность в том что положительные значения содержаться только у людей с опредленным типом занятости, убедимся в этом выведя таблицу с уникальными значениями `income_type`:

In [19]:
bank_df.loc[bank_df['days_employed'] > 0, 'income_type'].value_counts() 

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Действительно, почти все положительные значения отходят к определенному типу занятости: `пенсионер` и 2 к статусу `безработный`, подсчитаем средний трудовой стаж в годах: 

In [20]:
bank_df.loc[bank_df['days_employed'] >0, 'days_employed'].mean() / 365

1000.0118079897758

1000 лет - значение явно не соответствует действительности. 
Сделаем предположение о том, что трудовой стаж содержит значение в другой системе исчестелния, например, в часах. Проверим предположение, преобразова дни в часы:

In [21]:
(bank_df.loc[bank_df['days_employed'] >0, 'days_employed'].mean() / 24) / 365 

41.66715866624065

Средний стаж для пенсионеров 41 год, вполне удвлитворительное значение учитывая, то что люди в России выходят на пенсию в среднем в 60-63 года

*Выдвнем предположение об ошибках в значениях общего трудового стажа: вероятнее всего данные бьли собраны из двух и более разных источников и не были приведену к общему виду*

Заменим некорретный значения в датафрейме, согласно выдвинутым предположениям:
* отрицательные значения заменим на положительные
* положительные преобразуем в сутки, разделив на 24

In [22]:
#сначала разделим положительные значения
bank_df.loc[bank_df['days_employed'] > 0 ,'days_employed'] /= 24

In [23]:
#затем поменяем все отрицательные значения на положительные
bank_df.loc[bank_df['days_employed'] < 0 ,'days_employed'] *= (-1)

Проверим значения по первым 5ти строкам кода:

In [24]:
bank_df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Обработка анамалий в столбце дети

Проверим какие уникальные значения создержит столбец с детьми:

In [25]:
# к столбцу 'children' применим функцию value_counts()
bank_df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружили 2 анамалии: отрицательное значение и сильно выдающеесе значение в колличестве 20 детей.

*По аналогии со столбцом `days_employed` выдвигаем гипотезу о том, что знаения были занесены изначально некорретно, отрицательное значение было добавлено по ошибке, а в колличесте 20 детей - лишний ноль.*

Заменим данные значение:

In [26]:
# применив, атрибут .loc выполним арифметическую опеарцию по замене значений
bank_df.loc[bank_df['children'] == -1 ,'children'] *= (-1)
bank_df.loc[bank_df['children'] == 20 ,'children'] = 2

Проверим, произвелась ли замена:

In [27]:
bank_df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Обработка анамалий в столбце  возраст клиента

Проверим распределение значений по возрасту:

In [28]:
# для проверки распредления используем метод describe()
bank_df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Обнаружена анамалия в виде минимально значения. Очевидно возраст не может быть равен нулю, а также в данных о задолженностях по кредиту не могут быть люди младше 18 лет 

Вывдем колличество строк для двух условий:

In [29]:
# методом .loc выберем нужный столбец и соответствующие условия
print(len(bank_df.loc[bank_df['dob_years'] == 0]))
print(len(bank_df.loc[bank_df['dob_years'] < 18]))

101
101


Значения равны, поэтому анамалия в виде минимальных значений носит однородный характер, проверим взаимосвязь с другими элементами таблицы:

In [30]:
#bank_df.loc[bank_df['dob_years'] == 0].head(15)

Взаимосвязи с дургими элемнтами не было обнаружено, *выдвинем следющую гипотезу о появление нулевых значений: "Ноль был поставлен намерено, что бы обозначить, то что данных о возрасте либо нет, либо они были утеряны"*

Заменим нулевые значения на среднее для соответствующей группы. Выведем среднее значение возраста для группы 'пенсионеры' и для всех остальных:

In [31]:
# методом .loc отсортируем значения для двух групп и применим к группам метож .mean() 
retiree_mean = bank_df.loc[bank_df['income_type'] == 'пенсионер', 'dob_years'].mean()
print('Средний возраст для пенсионеров:', retiree_mean)
not_retiree_mean = bank_df.loc[bank_df['income_type'] != 'пенсионер', 'dob_years'].mean()
print('Средний возраст для всех остальных клиентов:', not_retiree_mean)


Средний возраст для пенсионеров: 59.06301867219917
Средний возраст для всех остальных клиентов: 39.85188748655838


Заменим нулевые значения для соответствующих групп:

In [32]:
# в методе .loc применим два условия и заменим на соответствующие средние значения
bank_df.loc[(bank_df['income_type']=='пенсионер') & (bank_df['dob_years'] ==0), 'dob_years'] = retiree_mean

bank_df.loc[(bank_df['income_type']!='пенсионер') & (bank_df['dob_years'] ==0), 'dob_years']= not_retiree_mean

Проверим выполнилась ли замена:

In [33]:
len(bank_df.loc[bank_df['dob_years'] == 0])

0

#### Обработка анамалий в столбце  пол клиента

Для поиска возможных аномалий вывдем уникальные значения для пола клиенты:

In [34]:
bank_df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Обнаружена анамалия в 1й строке, в виде неопредленного пола:

In [35]:
bank_df.loc[bank_df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


*Значение `XNA` мы не может отнести к какому либо полу, данные были занесены некорркектно, т.к. такая строка всего лишь одна - удалим её:*

In [36]:
# Методом drop.() удалим нужную строку по номеру индекса
bank_df.drop(labels = [10701], axis=0, inplace = True)

Проверим осталась ли анамалия в столбце `gender`

In [37]:
bank_df['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

### Выводы:  <a class="tocSkip">
    
1. Обнаружили анамалии в столбцах `children`, `days_employed` `dob_years` и `gender`
2. Выдвинули гипотезы их происхождения - для всех столбцов - это ошибки в занесении данных
3. Устронили ошибки в данных

### Замена типа данных 

В стобцах `days_employed` и `total_income`  используются значения float64, которые содержат значения после запятой, для удобства чтения таблицы и дальнейших операций, заменим на целочисленные значения 

In [38]:
bank_df['total_income'] = bank_df['total_income'].astype('int')
bank_df['days_employed'] = bank_df['days_employed'].astype('int')


Столбец с значением о данных кредитах, заменим на булевое значение, после чего:
* 0 - будет возвращать значение `False` - нет задолженности,
* 1 - будет возвращать значение `True`- есть задолженность

In [39]:
bank_df['debt'] = bank_df['debt'].astype('bool')

Проверим, заменились ли значения:

In [40]:
bank_df.dtypes

children              int64
days_employed         int32
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                   bool
total_income          int32
purpose              object
dtype: object

### Выводы:  <a class="tocSkip">
Заменили типы значения для 3х столбцов, таблицу стало читать проще благодаря убранным знакам после запятой, не несущих в себе особо важной информации, а замена значение о задолженности в таблице теперь показывает её наличае в виде переменной True и False

### Обработка дубликатов

Для начала обработаем таблицу на налчие неявныхых дубликатов.

В столбце `education` встречаются дубликаты которые содержат одну информацию но записаны в разных регистрах, посмотрим на все уникальные значения:

In [41]:
bank_df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем все значения к нижнему регистру:

In [42]:
# применим. str.lower() что бы привести к нижнему регистру все значения в столбце
bank_df['education'] = bank_df['education'].str.lower()
bank_df['education'].value_counts()                     

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

Повторим такую же операцию, для столбца `family_status`, в нём нет, неявных будликатов, но значение `Не женат / не замужем` написано с большой буквы, что нарушает общий стиль:

In [43]:
bank_df['family_status'].value_counts()      

женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Исправим это:

In [44]:
bank_df['family_status'] = bank_df['family_status'].str.lower()
bank_df['family_status'].value_counts()     

женат / замужем          12380
гражданский брак          4176
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

После приведния к нижнему регистру посчитаем колличество явных дубликатов в таблице:

In [45]:
# для всего датафреймв вызовим последовательно функции duplicated(). и sum()
bank_df.duplicated().sum()

71

*Приведение к нижнему регистру полностью очистило столбец от неявных дубликатов, здесь явная ошибка при записи данных: если данные указывает сам человек, например на сайте, форма "разрешает" заполнить её в любом виде и дальше данные не обрабатываются и попадают в датафрейм в таком виде, также сотрудники банка сами могут заполнять в неверном виде, самое простое решение проблемы обратиться в отдел разработки, для создания фильтра который будет обрабатывать такие значения до того как они попабудт в датафрейм*

Обработаем явные дублиаты в таблице:

In [46]:
bank_df = bank_df.drop_duplicates().reset_index(drop=True) 

Проверим таблицу на наличие дубликатов:

In [47]:
bank_df.duplicated().sum()

0

### Выводы:  <a class="tocSkip">
В таблице устранили дубликаты: теперь наглядно видны 5 групп в столбце об образовании, выдвинули гипотезу о проихождении дубликатов, предложили варианты решения будующих проблем 

### Оптимизация таблицы

Для упращения визуальной работы с таблицей и уменьшения размера общего файла классифицируем данные в столлбцах об образовании и семеном положении
Для начала разделим таблицу на две части:

In [48]:
# создадим новый датфрейм с названием education_glossary
# выведем на экран первые 5 строк 
education_glossary = bank_df[['education_id', 'education']]
education_glossary.head()


,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее


Выше полуили таблицу для значений об образовании, повтроим аналогичное действие для столбца о семейном положении:

In [49]:
# создадим новый датфрейм с названием family_glossary
# выведем на экран первые 5 строк 
family_glossary = bank_df[['family_status_id', 'family_status']]
family_glossary.head()

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак


Получили 2 датафрейма, что бы завершить создания из них словарей для общей таблицы, удалим из них дубликаты
Для данных об образовании:

In [50]:
education_glossary = education_glossary.drop_duplicates().reset_index(drop=True)
education_glossary

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Для данных о семейном положении:

In [51]:
family_glossary = family_glossary.drop_duplicates().reset_index(drop=True)
family_glossary

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Получили 2 'словаря', содержащие по 4 значения, что значительно сокращает объём занимаемой памяти в таблице, из исходной таблицы удалим значения `education` и `family_status`, оставив только их иденнтификаторы

In [52]:
# применим метод drop, для axis установим знаечние - 1 для удаления всего столбцы
bank_df = bank_df.drop('education', 1)
bank_df = bank_df.drop('family_status', 1)

Проверим, какие колонки остались в таблице после удаления и колличество занимаемой памяти: 

In [53]:
bank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     21453 non-null  int32  
 2   dob_years         21453 non-null  float64
 3   education_id      21453 non-null  int64  
 4   family_status_id  21453 non-null  int64  
 5   gender            21453 non-null  object 
 6   income_type       21453 non-null  object 
 7   debt              21453 non-null  bool   
 8   total_income      21453 non-null  int32  
 9   purpose           21453 non-null  object 
dtypes: bool(1), float64(1), int32(2), int64(3), object(3)
memory usage: 1.3+ MB


Проверим, как теперь отображается таблица:

In [54]:
display(bank_df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,0,0,F,сотрудник,False,253875,покупка жилья
1,1,4024,36.0,1,0,F,сотрудник,False,112080,приобретение автомобиля
2,0,5623,33.0,1,0,M,сотрудник,False,145885,покупка жилья
3,3,4124,32.0,1,0,M,сотрудник,False,267628,дополнительное образование
4,0,14177,53.0,1,1,F,пенсионер,False,158616,сыграть свадьбу


### Выводы:  <a class="tocSkip">
Оптимизировали таблицу:
1. Таблица стала "компактнее", данные об образовании и семейном статусе, теперь содержат только номера id, при необходимости соотвтствие номеру можно найти в созданных словарях 
2. Сокращён объём занимаемой памяти, т.к. были удален строковые значения 

### Категоризация данных

#### Категоризация по ежемесячному доходу

Для удобства анализа данных по статье ежемесячного дохода установим следующие группы:
* 0–30000 — `E`;
* 30001–50000 — `D`;
* 50001–200000 — `C`;
* 200001–1000000 — `B`;
* 1000001 и выше — `A`.

In [55]:
# объвили функцию total_income_group, которая содержит значиния total_income
# в теле функции 4 условия, согласно которым возвращается соответствующее значение
def total_income_group(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    else:
        return 'A'

Проверим роботоспрособность функции для каждого дипазон

In [56]:
print(total_income_group(100))
print(total_income_group(35000)) 
print(total_income_group(55500)) 
print(total_income_group(500000)) 
print(total_income_group(2000000)) 

E
D
C
B
A


Функция работает корретно для всех значений, добавим столбец с значениями в таблицу:

In [57]:
bank_df['total_income_group'] = bank_df['total_income'].apply(total_income_group)

In [58]:
display(bank_df.head()) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_group
0,1,8437,42.0,0,0,F,сотрудник,False,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,False,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,False,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,False,267628,дополнительное образование,B
4,0,14177,53.0,1,1,F,пенсионер,False,158616,сыграть свадьбу,C


<div class="alert alert-block alert-success">
<b>Комментарий ревьюера✅:</b> Функция написана верно, круто что результат ее выполнения проверен👍

#### Категоризация по целям кредита

В столбце цели кредита есть множество значений, которые содержат значения которые можно отнести к дубликатам, их можно разделить на 4 категории: 
* `операции с автомобилем`;
* `операции с недвижимостью`;
* `проведение свадьбы`; 
* `получение образования`.

Посмотрим какие уникальные значение содержит столбец `purpose`: 

In [59]:
bank_df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      620
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Созадим функцию которая будет возвращать значения в отдельный столбец с категориями:

In [60]:
# объявяили функцию "purpose_category" которая содержит значения "purpose"
# в теле функции 4 услвоия которые проверют значения по корню слова
# и 5е значение для неопределенных значений 
def purpose_category(purpose): 
    if ('вадьба' in purpose or
        'ыграть' in purpose or
        'ровести'in purpose or
       'свадьбы' in purpose):
        return 'сыграть свадьбу'
    if ('авто' in purpose or
       'машин' in purpose):
        return 'операции с автомобилем'
    if ('образ' in purpose or
       'обуч' in purpose or
       'учёб' in purpose or
       'учеб' in purpose):
        return 'получение образования'
    if ('ремонт' in purpose or
       'недвиж' in purpose or
       'дом' in purpose or
       'квартира' in purpose or
       'жил' in purpose):
        return 'операции с недвижимостью'
    else:
        return 'категория не опредлена'

Проверим работы функции для некоторых значение:

In [61]:
print(purpose_category('на проведение свадьбы ')) 
print(purpose_category('сделка с подержанным автомобилем'))  
print(purpose_category('обучение в Яндекс.Практикуме')) 
print(purpose_category('полёт в космос')) 

сыграть свадьбу
операции с автомобилем
получение образования
категория не опредлена


По анаалогии с категоризацией по ежемесячному доходу, добавим новый столбец `purpose_category`:

In [62]:
bank_df['purpose_category'] = bank_df['purpose'].apply(purpose_category)

Проверим есть ли строки с неопределенной категорией: 

In [63]:
bank_df.loc[bank_df['purpose_category'] == 'категория не опредлена', 'purpose']

Series([], Name: purpose, dtype: object)

Все указаные цели были отнесены к заданным категориям, т.к. нет значний с непоределенной категорией,  выведем итоговую таблицу на экран:

In [64]:
display(bank_df.head()) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_group,purpose_category
0,1,8437,42.0,0,0,F,сотрудник,False,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,False,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,False,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,False,267628,дополнительное образование,B,получение образования
4,0,14177,53.0,1,1,F,пенсионер,False,158616,сыграть свадьбу,C,сыграть свадьбу


### Выводы:  <a class="tocSkip">
Добавили в таблицу 2 столбца, которые относят каждого клиента к определнной категори: таблица стала более информативна, а также пояивилась возможнсоть сравнивать данные по опредленным группам, находить закономерности в них и проч. возможности 

### Вывод по разделу:  <a class="tocSkip">
    
Данные успешно обработны, можно приступать к ответам на вопросы

## Ответы на вопросы:

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для наглядной демонстрации значений создадим сводные таблицы

In [65]:
#создадим сводную таблицу методом pivot_table() 
#выведем таблицу на экран
pivot_children = bank_df.pivot_table(index='children', values= 'debt',  aggfunc=['count', 'mean'])
pivot_children

,count,mean
,debt,debt
children,,
0,14090,0.075444
1,4855,0.091658
2,2128,0.094925
3,330,0.081818
4,41,0.097561
5,9,0.000000


**Ответ:**

Исходя, из сводной таблицы прослеживается следующая закономерность: клиенты не имеющие детей -  реже имеют задолженности, чем клиенты с хотя бы одним ребёнком

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [66]:
# по аналогии со свдной таблицей df_pivot_children
pivot_family_status = bank_df.pivot_table(index='family_status_id', values= 'debt',  aggfunc=['count', 'mean'])
pivot_family_status =  pivot_family_status.merge(family_glossary, on='family_status_id', how='left')
pivot_family_status = pivot_family_status.drop('family_status_id', 1)
pivot_family_status

,"(count, debt)","(mean, debt)",family_status
0,12339,0.075452,женат / замужем
1,4150,0.093494,гражданский брак
2,959,0.065693,вдовец / вдова
3,1195,0.071130,в разводе
4,2810,0.097509,не женат / не замужем


**Ответ:**

Данные из полученной таблицы говорят о том, что люди с семейным статусом `вдовец / вдова` менее часто имеют задолженности по кредитным обязательствам, немного надёжнее клиенты состоящие или некогда состоявшие в браке(`женат/замужем` , `в разводе`), а в большей мере сталкиваются с задолженностью - клиенты с официально не зарегстрированом семейным положением(`гражданский брак`, `не женат / не замужем`)  

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [67]:
# по аналогии со свдной таблицей df_pivot_children
bank_df.pivot_table(index='total_income_group', values= 'debt',  aggfunc=['count', 'mean']) 

,count,mean
,debt,debt
total_income_group,,
A,25,0.080000
B,5040,0.070635
C,16016,0.084915
D,350,0.060000
E,22,0.090909


**Ответ:**

Исходя из сводной таблицы основная масса заёмщиков, имеет доходы:
* `B`— 200000–1000000 
* `C`  — 50000–200000

Клиенты из категории `B` менее часто сталкиваются с задолженностью по кредиту, соответствуенно делаем вывод о том, что люди с большим доходом более ответственно относятся к соблюдению выплат по кредитным обязетельствам

### Как разные цели кредита влияют на его возврат в срок?

In [68]:
# по аналогии со свдной таблицей df_pivot_children
bank_df.pivot_table(index='purpose_category', values= 'debt',  aggfunc=['count', 'mean'])  

,count,mean
,debt,debt
purpose_category,,
операции с автомобилем,4306,0.093590
операции с недвижимостью,10810,0.072340
получение образования,4013,0.092200
сыграть свадьбу,2324,0.080034


**Ответ:** 

Клиенты, берущие кредит на цели `операции с автомобилем` и 	`получение образования` имеют задолженности чаще чем остальные.

Среднее значение занимают люди, берущие кредит с целью `сыграть свадьбу`.

Группа клиентов, берущая кредит для `операций с недвижимостью` самая многоичесленная, а также составляет меньшую часть по наличию задолженностей	

### Вывод по разделу:  <a class="tocSkip">

В построенных сводных таблицах нет анамально выделяющихся значений, поэтому можем сделать вывод о том, что данные были обработаны корректно. 
    
На факт возрата кредита в срок влияеют: колличество детей у клиента, его семейное положение, доход и цель с которой он его берет, по-мимио этого, на основе таблиц можно выделить группы по их колличеству, например:
* клиентов, не имеющих детей намного больше чем с хотя бы одним ребенком;
* женатых/замужних клиентов намного больше чем всех остальных;
* клиенты имеют доходы от 50 до 200 тыс. и от 200 до 1 млн.;
* чаще всего клиенты берут кредит на операции с недвжимиостью.   

## Итоги исследования

В данном проекте было выполнена:

Работа по предобработке данных: 
* в ней были заполнены пропущенный значения в таблице на основании гипотезы о том, что данные были утеряны или некорретно зансены
* обработаны анамалии в виде неправдоподобных значений, выдвинуты гипотезы об их происхождении
* таблица была оптимизирована и готова для хранения больших объёмов данных
* были опредлены категории по доходам и целям, для формирования больших групп внутри таблицы, написанные функции автомтически оперделят новые записи к той или иной группе 

Ответы на поставленные вопросы:
* для ответа на поставленные вопрсы были построены сводные таблицы
* на основе данных таблицах был выполнил анализ по зависимости того или иного фактора на факт выплаты кредита в срок
* для каждой группы были обнаружены закономерности и сделаны подробные выводы

По вопросам зависосмоти от того или иного фактора можно выявить следующий тенденции:
* люди не имеющие детей менее часто имеют задолженности;
* клиенты не состощие в официально зарегестрированных отношениях чаще всех имеют задолженности;
* чем больше доход у клиента, тем меньше вероятность у него появления задолженности;
* менее часто имеют задолженности клиенты, взявшие его с целью покупки/строительства и прочих операций с неждвижимостью.